### Barrier-Free Predict Flask App Run for Colab

In [0]:
!pip install flask-ngrok

In [0]:
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/python/barrierfree-server-run')

from barrier_free_model import BFModel, process_img, decode_predictions

---

In [0]:
# 앙상블 사용 여부
is_ensemble = False

In [0]:
# 사용할 모델
PATH = '/content/gdrive/My Drive/python/ensemble/models/'

MODEL_PATH = PATH + '0531-K-fold--gap-densenet201-size299-noise70-N3-fold-3-best-acc.h5'
TARGET_SIZE = (299, 299)

ENSEMBLE_LIST = [PATH + '0529-K-fold-gap-densenet201-size299-N3_epoch15-fold-4_lastEpoch.h5',
                 PATH + '0530-K-fold-gap-densenet201-size299-N3_epoch15-fold-2_lastEpoch.h5',
                 PATH + '0531-K-fold--gap-densenet201-size299-noise70-N3-fold-3-best-acc.h5',
                 PATH + '0601-K-fold--gap-vgg19-N3-2try-fold-0-best-acc.h5']
TARGET_SIZE_LIST = [(299, 299), (299, 299), (299, 299), (224, 224)]
NUM_ENSEMBLE = 4

---

In [0]:
import io
import argparse
import numpy as np
from PIL import Image
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
run_with_ngrok(app)

model = None


def load_defined_model():

    global model
    print('* Loading model')

    if is_ensemble:
        print('* Ensemble mode')
        model = []
        for m in ENSEMBLE_LIST:
            model.append(BFModel(m))
    else:
        print('* Single mode')
        model = BFModel(MODEL_PATH)
        
    print('* Model is loaded')


@app.route('/predict', methods=['POST'])
def image_classifier():

    image = request.files['image'].read()
    image = io.BytesIO(image)
    image = Image.open(image)

    if is_ensemble:
        pred = np.zeros(shape=(1, 2))
        for idx, ts in enumerate(TARGET_SIZE_LIST):
            _image = process_img(image, ts)
            pred += model[idx].predict(_image)
        pred /= NUM_ENSEMBLE
    else:
        image = process_img(image, TARGET_SIZE)
        pred = model.predict(image)

    result, y_pred = decode_predictions(pred)

    result_dict = {'result': []}

    for r in result:
        result_dict['result'].append(r)

    return jsonify(result_dict)


def run_app():

    load_defined_model()
    print('* Starting server')
    app.run()

`Running on http://xxxxxxxxxxxx.ngrok.io` 링크에 POST 요청을 하면 됩니다.

In [0]:
run_app()